In [108]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date
from sqlalchemy.orm import relationship, backref, declarative_base, sessionmaker
from faker import Faker
import random
from datetime import datetime

Create a declarative base class, a faker instance and the file of the database

In [109]:
Base = declarative_base()

fake = Faker(['es_MX'])
engine = create_engine('sqlite:///DummyDB.db', echo=True)

Now create al the classes that defines the tables of our db

In [110]:
class Users(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)

    last_name = Column(String)
    first_name = Column(String)
    email = Column(String)
    address = Column(String)
    city = Column(String)
    state = Column(String)
    zip_code = Column(String)
    user_role = Column(String)

    ############################################################################
    ####    I tried to create the relations but had some problems when I    ####
    ####        was trying to fill the tables with the id references        ####
    ####            from the other tables so I commented them.              ####
    ############################################################################

    #performance = relationship('Performance', backref=backref('users', lazy=True))

    def __repr__(self):
        return f'<User(name={self.first_name}, email={self.email})>'
    
class Performance(Base):
    __tablename__ = 'performance'

    perf_id = Column(Integer, primary_key=True)

    date = Column(Date)
    ensemble_id = Column(Integer)#, ForeignKey('ensemble.ensemble_id'))
    tab_id = Column(Integer)#, ForeignKey('scores.tab_id'))
    user_id = Column(Integer)#, ForeignKey('users.id'))

class Scores(Base):

    __tablename__ = 'scores'

    tab_id = Column(Integer, primary_key=True)

    improvisation = Column(Integer)
    style = Column(Integer)
    presentation = Column(Integer)
    musicality = Column(Integer)
    ensemble = Column(Integer)
    intonation = Column(Integer)
    technique = Column(Integer)

    #performance = relationship('Performance', backref=backref('scores', lazy=True))

class School(Base):
    
    __tablename__ = 'school'

    school_id = Column(Integer, primary_key=True)

    school_name = Column(String)
    phone_number = Column(String)
    school_address = Column(String)
    zip_code = Column(String)
    city = Column(String)
    state = Column(String)

    #Define a relationship to the Ensemble class
    #ensemble = relationship('Ensemble', backref=backref('school', lazy=True))

    def __repr__(self):
        return f'<User(school_name={self.school_name}, phone={self.phone_number})>'    
    
class Director(Base):
    __tablename__ = 'director'

    director_id = Column(Integer, primary_key=True)

    last_name = Column(String)
    first_name = Column(String)
    phone = Column(String)
    email = Column(String)

    #Define a relationship to the Ensemble class
    #ensemble = relationship('Ensemble', backref=backref('director', lazy=True))

    def __repr__(self):
        return f'<director_name={self.first_name} {self.last_name}, phone={self.phone}, email={self.email}>' 

class Ensemble(Base):
    
    __tablename__ = 'ensemble'

    ensemble_id = Column(Integer, primary_key=True)

    class_ = Column(String)
    number_in_ensemble = Column(Integer)
    school_id = Column(Integer)#, ForeignKey('school.school_id'))
    director_id = Column(Integer)#, ForeignKey('director.director_id'))

    #Define a relationship to the Performance class
    #performance = relationship('Performance', backref=backref('ensemble', lazy=True))       


Now we create the connection to the database and load the scheme of it.

In [111]:
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

2023-03-01 12:53:36,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-01 12:53:36,725 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-03-01 12:53:36,726 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-01 12:53:36,727 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-03-01 12:53:36,727 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-01 12:53:36,728 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("performance")
2023-03-01 12:53:36,728 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-01 12:53:36,729 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("performance")
2023-03-01 12:53:36,730 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-01 12:53:36,731 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("scores")
2023-03-01 12:53:36,731 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-01 12:53:36,732 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("scores")
2023-03-01 12:53:36,733 INFO sqlalchemy.engine.Engine [raw sql] 

Here we fill de tables using neither random data or faker's one, also I included some lists with things that faker does not generate.

I created a list for each entry and then insert them to its respectively table.

In [112]:
classes = ['music', 'acting', 'harmony']
role = ['student', 'professor']

users = [Users(
    last_name=fake.last_name(),
    first_name = fake.first_name(),
    email=fake.email(),
    address=fake.city(),
    city=fake.city(),
    state=fake.state(),
    zip_code=fake.postcode(),
    user_role=random.choice(role)
) for _ in range(5)]
session.add_all(users)
#conn.execute(Users.__table__.insert(), [u.__dict__ for u in users]) 

########################################################################################
####    Here i tried to use this commands to insert into the tables, but i failed   #### 
####         so I used  the method add_all() and it worked well                     ####  
########################################################################################

school = [School(
    school_name = f'Colegio {random.randint(1,100)}',
    phone_number = fake.phone_number(),
    school_address = fake.address(),
    zip_code = fake.postcode(),
    city = fake.city(),
    state = fake.state()
) for _ in range(10)]
session.add_all(school)
#conn.execute(School.__table__.insert(), [sc.__dict__ for sc in school])

director = [Director(
    last_name = fake.last_name(),
    first_name = fake.first_name(),
    phone = fake.phone_number(),
    email = fake.email()
)for _ in range(7)]
session.add_all(director)
#conn.execute(Director.__table__.insert(), [d.__dict__ for d in director])

scores = [Scores(
    improvisation = random.randint(5, 11),
    style = random.randint(5, 11),
    presentation = random.randint(5, 11),
    musicality = random.randint(5, 11),
    ensemble = random.randint(5, 11),
    intonation = random.randint(5, 11),
    technique = random.randint(5, 11)
) for _ in range(4)]
session.add_all(scores)
#conn.execute(Scores.__table__.insert(), [s.__dict__ for s in scores])

ensemble = [Ensemble(
    class_ = random.choice(classes),
    number_in_ensemble = random.randint(0,10),
    school_id = random.randint(0,10),#school[i % 10],
    director_id = random.randint(0,7)#director[i % 10]
) for _ in range(10)]
session.add_all(ensemble)
#conn.execute(Ensemble.__table__.insert(), [e.__dict__ for e in ensemble])

################################################################################
#### In the ensemble list we can see the attempt to create an entry using   ####
####     the index of the table that i tried to reference, but for now      ####
####            i decided to not create relationships between them          ####
################################################################################

# 10 entries in performance
performance = [Performance(
    date=datetime.strptime(fake.date(pattern='%d-%m-%Y %H:%M:%S'), '%d-%m-%Y %H:%M:%S'),
    ensemble_id=random.randint(0,10),#ensemble[i % 10],
    tab_id=random.randint(0,4),#scores[i % 10],
    user_id=random.randint(0,5)#users[i % 10]
) for _ in range(5)]
session.add_all(performance)
#conn.execute(Performance.__table__.insert(), [p.__dict__ for p in performance])

session.commit() #this command updates the changes in de db file

2023-03-01 12:53:36,817 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-01 12:53:36,826 INFO sqlalchemy.engine.Engine INSERT INTO director (last_name, first_name, phone, email) VALUES (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?) RETURNING director_id
2023-03-01 12:53:36,830 INFO sqlalchemy.engine.Engine [generated in 0.00014s (insertmanyvalues)] ('Gamboa', 'Ivonne', '231-676-2867', 'mercedes36@example.com', 'Mora', 'Aurora', '(715)589-8943x3237', 'barbara27@example.net', 'Gaitán', 'Emiliano', '080-113-9507', 'hector85@example.com', 'Oquendo', 'Isabel', '05995267099', 'maria41@example.com', 'Concepción', 'Trinidad', '(175)709-5780x54521', 'elisaaguirre@example.org', 'Solano', 'Leonardo', '1-429-454-0764x52110', 'walonzo@example.com', 'Salas', 'Ana Luisa', '1-557-907-4275x6315', 'rosariobriseno@example.com')
2023-03-01 12:53:36,834 INFO sqlalchemy.engine.Engine INSERT INTO ensemble (class_, number_in_ensemble, school_id, director

In [113]:
# Some queries
result1 = session.query(Users).all()
for r in result1:
    print(r) 

2023-03-01 12:53:36,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-01 12:53:36,897 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.last_name AS users_last_name, users.first_name AS users_first_name, users.email AS users_email, users.address AS users_address, users.city AS users_city, users.state AS users_state, users.zip_code AS users_zip_code, users.user_role AS users_user_role 
FROM users
2023-03-01 12:53:36,898 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
<User(name=Soledad, email=crangel@example.org)>
<User(name=Juan Carlos, email=esteban93@example.org)>
<User(name=Miguel Ángel, email=hugozaragoza@example.com)>
<User(name=Genaro, email=almabustos@example.net)>
<User(name=Sara, email=raul11@example.net)>
